In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/movies_metadata.csv'
df = pd.read_csv(path)

<ipython-input-15-6851e4b1568b>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
ndf = df[["title", "overview"]]
ndf

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
45461,Subdue,Rising and falling between a man and woman.
45462,Century of Birthing,An artist struggles to finish his work while a...
45463,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,Satan Triumphant,"In a small town live two brothers, one a minis..."


In [ ]:
ndf.isnull().sum()

,0
title,6
overview,954


In [ ]:
ndf.dropna()
ndf.isnull().sum()

,0
title,6
overview,954
combined_text,0


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
ndf['combined_text'] = ndf['title'] + " " + ndf['overview']
ndf

<ipython-input-68-ee1742c3d530>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['combined_text'] = ndf['title'] + " " + ndf['overview']


,title,overview,combined_text
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Toy Story Led by Woody, Andy's toys live happi..."
1,Jumanji,When siblings Judy and Peter discover an encha...,Jumanji When siblings Judy and Peter discover ...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpier Old Men A family wedding reignites th...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Waiting to Exhale Cheated on, mistreated and s..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Father of the Bride Part II Just when George B...
...,...,...,...
45461,Subdue,Rising and falling between a man and woman.,Subdue Rising and falling between a man and wo...
45462,Century of Birthing,An artist struggles to finish his work while a...,Century of Birthing An artist struggles to fin...
45463,Betrayal,"When one of her hits goes wrong, a professiona...","Betrayal When one of her hits goes wrong, a pr..."
45464,Satan Triumphant,"In a small town live two brothers, one a minis...",Satan Triumphant In a small town live two brot...


In [ ]:
import time

# Dictionary to store start times for multiple timers
timers = {}

def start_timer(label="default"):
    """Start a timer with a specific label."""
    timers[label] = time.time()
    print(f"Timer '{label}' started.")

def end_timer(label="default"):
    """End a timer with a specific label and print the elapsed time."""
    if label not in timers:
        print(f"No timer found with label '{label}'.")
        return

    elapsed_time = time.time() - timers[label]
    print(f"Timer '{label}' ended. Elapsed time: {elapsed_time:.4f} seconds.")
    # Optionally, remove the timer entry to reset it
    del timers[label]

In [ ]:
ndf['combined_text'] = ndf['combined_text'].astype(str)

<ipython-input-69-27daa6b7d044>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['combined_text'] = ndf['combined_text'].astype(str)


In [ ]:
start_timer("Encode movie overviews")
ndf['embedding'] = ndf['combined_text'].apply(lambda x: model.encode(x))
end_timer("Encode movie overviews")

Timer 'Encode movie overviews' started.
Timer 'Encode movie overviews' ended. Elapsed time: 2513.0473 seconds.


<ipython-input-70-0b871b561d87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['embedding'] = ndf['combined_text'].apply(lambda x: model.encode(x))


In [ ]:
import pickle

with open('encoded_movies.pkl', 'wb') as f:
    pickle.dump(ndf, f)

In [ ]:
with open('encoded_movies.pkl', 'rb') as f:
    df = pickle.load(f)

print("Embeddings loaded successfully!")

Embeddings loaded successfully!


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search_movies(query, ndf, top_k=5):
    # Encode the query
    query_embedding = model.encode(query)

    # Compute cosine similarity
    similarities = cosine_similarity([query_embedding], ndf['embedding'].tolist())[0]

    # Get top-k results
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    # Create a DataFrame with titles, overviews, and similarity scores
    results = ndf.iloc[top_indices][['title', 'overview']].copy()
    results['similarity_score'] = similarities[top_indices]

    return results

In [ ]:
# User query
query = "A movie about gambling"

# Search for relevant movies
results = search_movies(query, ndf)
print(results)

                      title  \
15                   Casino   
363                Maverick   
785        American Buffalo   
356  It Could Happen to You   
96                 Shopping   

                                              overview  similarity_score  
15   The life of the gambling paradise – Las Vegas ...          0.598285  
363  Maverick is a gambler who would rather con som...          0.478307  
785  Three inner-city losers plan a robbery of a va...          0.401414  
356  Charlie Lang is a simple, kindhearted New York...          0.394825  
96   A dark, hip, urban story of a barren and anony...          0.385935  
